### requirements for the following codings


In [ ]:
### packages required 
!pip install fair-esm 
!pip install torch
!pip install tensorflow
!pip install sklearn

### peptide embeddings with esm2_t6_8M_UR50D pretrained models
6 layers, 8M parameters, dataset: UR50/D 2021_04, embedding dimension: 320
mode download URL: https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt

In [1]:
def esm_embeddings(peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you conputer might automatically kill the job.
  import torch
  import esm
  import collections
  # load the model
  # NOTICE: if the model was not downloaded in your local environment, it will automatically download it.
  model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
  batch_converter = alphabet.get_batch_converter()
  model.eval()  # disables dropout for deterministic results

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = model(batch_tokens, repr_layers=[6], return_contacts=True)  
  token_representations = results["representations"][6]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  return embeddings_results

### data loading and embeddings (main dataset)

In [3]:
import numpy as np
import pandas as pd

In [17]:
# training dataset loading
dataset = pd.read_excel('AMP_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('AMP_train_main_esm2_t6_8M_UR50D_unified_320_dimension.csv')
# loading the y dataset for model development 
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model

In [5]:
# test dataset loading
dataset = pd.read_excel('AMP_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('AMP_test_main_esm2_t6_8M_UR50D_unified_320_dimension.csv')


# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [18]:
y_train

array([1, 0, 0, ..., 0, 0, 1])

In [20]:
# assign the dataset 
X_train_data_name = 'AMP_train_main_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'AMP_test_main_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [21]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(13428, 320)
(8953, 320)
(13428,)
(8953,)


### Model architecture

In [22]:
def ESM_CNN(X_train, y_train, X_test, y_test):
  from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Conv1D
  from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, AveragePooling1D, MaxPooling1D
  from keras.models import Sequential,Model
  from keras.optimizers import SGD
  from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
  import keras
  from keras import backend as K
  inputShape=(320,1)
  input = Input(inputShape)
  x = Conv1D(128,(3),strides = (1),name='layer_conv1',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool1',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Conv1D(32,(3),strides = (1),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(64,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')
  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate 
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lrate = LearningRateScheduler(step_decay)

  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,restore_best_weights = True)

  # summary the callbacks_list
  callbacks_list = [ lrate , early_stop]

  model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                            epochs=200,callbacks=callbacks_list,batch_size = 64, verbose=1)
  return model, model_history

### 10-fold cross validation

In [23]:
#Implementing 10-fold cross validation
from sklearn.model_selection import KFold
k = 10 
kf = KFold(n_splits=k, shuffle = True, random_state=1)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []

for train_index , test_index in kf.split(y_train):
    X_train_CV , X_valid_CV = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y_train_CV , y_valid_CV = y_train.iloc[train_index] , y_train.iloc[test_index]
    model, model_history = ESM_CNN(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV)
    # confusion matrix 
    predicted_class= []
    predicted_protability = model.predict(X_valid_CV,batch_size=1)
    for i in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_valid_CV    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_valid_CV, predicted_protability[:,1])
    AUC_collecton.append(AUC)


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
189/189 [==============================] - 7s 34ms/step - loss: 0.3141 - accuracy: 0.9036 - val_loss: 0.6856 - val_accuracy: 0.5465 - lr: 0.1000
Epoch 2/200
189/189 [==============================] - 8s 43ms/step - loss: 0.1491 - accuracy: 0.9451 - val_loss: 0.4972 - val_accuracy: 0.9360 - lr: 0.1000
Epoch 3/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1266 - accuracy: 0.9532 - val_loss: 0.1946 - val_accuracy: 0.9442 - lr: 0.0600
Epoch 4/200
189/189 [==============================] - 8s 40ms/step - loss: 0.1186 - accuracy: 0.9554 - val_loss: 0.1196 - val_accuracy: 0.9523 - lr: 0.0600
Epoch 5/200
189/189 [==============================] - 11s 60ms/step - loss: 0.1147 - accuracy: 0.9554 - val_loss: 0.1273 - val_accuracy: 0.9494 - lr: 0.0600
Epoch 6/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1046 - accuracy: 0.9590 - val_loss: 0.1206 - val_accuracy: 0.9509 - lr: 0.0360
Epoch 7/200
189/189 [==============================] - 6s

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


189/189 [==============================] - 7s 34ms/step - loss: 0.2968 - accuracy: 0.9078 - val_loss: 0.8969 - val_accuracy: 0.3083 - lr: 0.1000
Epoch 2/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1544 - accuracy: 0.9450 - val_loss: 0.6776 - val_accuracy: 0.4483 - lr: 0.1000
Epoch 3/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1295 - accuracy: 0.9509 - val_loss: 0.1614 - val_accuracy: 0.9546 - lr: 0.0600
Epoch 4/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1201 - accuracy: 0.9553 - val_loss: 0.1082 - val_accuracy: 0.9598 - lr: 0.0600
Epoch 5/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1156 - accuracy: 0.9571 - val_loss: 0.1005 - val_accuracy: 0.9605 - lr: 0.0600
Epoch 6/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1059 - accuracy: 0.9609 - val_loss: 0.1018 - val_accuracy: 0.9620 - lr: 0.0360
Epoch 7/200
189/189 [==============================] - 6s 33ms/step - 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


...

189/189 [==============================] - 7s 33ms/step - loss: 0.6515 - accuracy: 0.9055 - val_loss: 0.3349 - val_accuracy: 0.9449 - lr: 0.1000
Epoch 2/200
189/189 [==============================] - 6s 32ms/step - loss: 0.1545 - accuracy: 0.9437 - val_loss: 0.1597 - val_accuracy: 0.9531 - lr: 0.1000
Epoch 3/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1313 - accuracy: 0.9516 - val_loss: 0.0990 - val_accuracy: 0.9672 - lr: 0.0600
Epoch 4/200
189/189 [==============================] - 6s 32ms/step - loss: 0.1212 - accuracy: 0.9552 - val_loss: 0.1175 - val_accuracy: 0.9613 - lr: 0.0600
Epoch 5/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1167 - accuracy: 0.9564 - val_loss: 0.0953 - val_accuracy: 0.9695 - lr: 0.0600
Epoch 6/200
189/189 [==============================] - 6s 32ms/step - loss: 0.1105 - accuracy: 0.9571 - val_loss: 0.0920 - val_accuracy: 0.9687 - lr: 0.0360
Epoch 7/200
189/189 [==============================] - 6s 32ms/step - 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


189/189 [==============================] - 7s 34ms/step - loss: 0.3205 - accuracy: 0.8921 - val_loss: 0.6495 - val_accuracy: 0.7273 - lr: 0.1000
Epoch 2/200
189/189 [==============================] - 6s 34ms/step - loss: 0.1517 - accuracy: 0.9446 - val_loss: 0.4645 - val_accuracy: 0.9218 - lr: 0.1000
Epoch 3/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1279 - accuracy: 0.9521 - val_loss: 0.1413 - val_accuracy: 0.9478 - lr: 0.0600
Epoch 4/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1214 - accuracy: 0.9544 - val_loss: 0.1241 - val_accuracy: 0.9516 - lr: 0.0600
Epoch 5/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1150 - accuracy: 0.9569 - val_loss: 0.1323 - val_accuracy: 0.9501 - lr: 0.0600
Epoch 6/200
189/189 [==============================] - 6s 32ms/step - loss: 0.1074 - accuracy: 0.9590 - val_loss: 0.1155 - val_accuracy: 0.9545 - lr: 0.0360
Epoch 7/200
189/189 [==============================] - 6s 33ms/step - 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


189/189 [==============================] - 7s 33ms/step - loss: 0.2718 - accuracy: 0.8958 - val_loss: 0.7775 - val_accuracy: 0.2891 - lr: 0.1000
Epoch 2/200
189/189 [==============================] - 6s 32ms/step - loss: 0.1496 - accuracy: 0.9459 - val_loss: 0.5343 - val_accuracy: 0.7809 - lr: 0.1000
Epoch 3/200
189/189 [==============================] - 6s 33ms/step - loss: 0.1274 - accuracy: 0.9521 - val_loss: 0.1885 - val_accuracy: 0.9501 - lr: 0.0600
Epoch 4/200
189/189 [==============================] - 6s 32ms/step - loss: 0.1225 - accuracy: 0.9544 - val_loss: 0.1273 - val_accuracy: 0.9508 - lr: 0.0600
Epoch 5/200
189/189 [==============================] - 6s 32ms/step - loss: 0.1134 - accuracy: 0.9565 - val_loss: 0.1184 - val_accuracy: 0.9508 - lr: 0.0600
Epoch 6/200
189/189 [==============================] - 6s 32ms/step - loss: 0.1035 - accuracy: 0.9610 - val_loss: 0.1188 - val_accuracy: 0.9531 - lr: 0.0360
Epoch 7/200
189/189 [==============================] - 6s 32ms/step - 

In [24]:
from statistics import mean, stdev
print(mean(ACC_collecton),'±',stdev(ACC_collecton))
print(mean(BACC_collecton),'±',stdev(BACC_collecton))
print(mean(Sn_collecton),'±',stdev(Sn_collecton))
print(mean(Sp_collecton),'±',stdev(Sp_collecton))
print(mean(MCC_collecton),'±',stdev(MCC_collecton))
print(mean(AUC_collecton),'±',stdev(AUC_collecton))


0.9657426097455148 ± 0.004514571878611231
0.9621574686457981 ± 0.006778052699792235
0.9705491127336013 ± 0.0038237537324600887
0.9537658245579947 ± 0.01335963358900246
0.9160115232473642 ± 0.011581990766664892
0.9907999875980019 ± 0.003279298842209914


In [ ]:
ACC_collecton

[0.9815950920245399]

### model evaluation in test dataset

In [27]:
# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
model, model_history = ESM_CNN(X_train, y_train, X_test , y_test)
# confusion matrix 
predicted_class= []
predicted_protability = model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
y_true = y_test    
from sklearn.metrics import confusion_matrix
import math
# np.ravel() return a flatten 1D array
TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton.append(ACC)
Sn_collecton.append(TP/(TP+FN))
Sp_collecton.append(TN/(TN+FP))
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton.append(MCC)
BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(y_test, predicted_protability[:,1])
AUC_collecton.append(AUC)

Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


210/210 [==============================] - 8s 37ms/step - loss: 0.3073 - accuracy: 0.9056 - val_loss: 0.6940 - val_accuracy: 0.3489 - lr: 0.1000
Epoch 2/200
210/210 [==============================] - 8s 36ms/step - loss: 0.1443 - accuracy: 0.9476 - val_loss: 0.4291 - val_accuracy: 0.9433 - lr: 0.1000
Epoch 3/200
210/210 [==============================] - 8s 37ms/step - loss: 0.1246 - accuracy: 0.9517 - val_loss: 0.1545 - val_accuracy: 0.9535 - lr: 0.0600
Epoch 4/200
210/210 [==============================] - 8s 37ms/step - loss: 0.1186 - accuracy: 0.9559 - val_loss: 0.1503 - val_accuracy: 0.9445 - lr: 0.0600
Epoch 5/200
210/210 [==============================] - 8s 38ms/step - loss: 0.1127 - accuracy: 0.9571 - val_loss: 0.1181 - val_accuracy: 0.9545 - lr: 0.0600
Epoch 6/200
210/210 [==============================] - 8s 37ms/step - loss: 0.1034 - accuracy: 0.9595 - val_loss: 0.1175 - val_accuracy: 0.9561 - lr: 0.0360
Epoch 7/200
210/210 [==============================] - 10s 49ms/step -

In [28]:
print(ACC_collecton[0])
print(BACC_collecton[0])
print(Sn_collecton[0])
print(Sp_collecton[0])
print(MCC_collecton[0])
print(AUC_collecton[0])

0.9623589858148107
0.9579101207100744
0.968032278088144
0.9477879633320048
0.907729512154961
0.9909900935111878


In [13]:
model.save('AMP_tensorflow_model',save_format = 'tf') 
!zip -r /content/AMP_tensorflow_model.zip /content/AMP_tensorflow_model

  adding: content/AMP_alternative_tensorflow_model/ (stored 0%)
  adding: content/AMP_alternative_tensorflow_model/assets/ (stored 0%)
  adding: content/AMP_alternative_tensorflow_model/keras_metadata.pb (deflated 89%)
  adding: content/AMP_alternative_tensorflow_model/variables/ (stored 0%)
  adding: content/AMP_alternative_tensorflow_model/variables/variables.index (deflated 64%)
  adding: content/AMP_alternative_tensorflow_model/variables/variables.data-00000-of-00001 (deflated 31%)
  adding: content/AMP_alternative_tensorflow_model/saved_model.pb (deflated 88%)
